<a href="https://colab.research.google.com/github/mrandrivan/ML-DL-AI-practice/blob/main/Genetics_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
import pandas as pd

data = pd.read_csv('GSE25066_series_matrix.txt', sep='\t', error_bad_lines=False)

# View the first few rows
print(data.head())


TypeError: read_csv() got an unexpected keyword argument 'error_bad_lines'